In [24]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import pandas as pd

from torch import nn

In [16]:
tokenizer = AutoTokenizer.from_pretrained("E://competition//atoms//ChemBERTa-77M-MTR")
model = AutoModelForMaskedLM.from_pretrained("E://competition//atoms//ChemBERTa-77M-MTR")

c:\Users\Admin\miniconda3\envs\crawl\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at E://competition//atoms//ChemBERTa-77M-MTR and are newly initialized: ['lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
data_train = pd.read_csv("../data/train.csv")

In [18]:
data_train.head()

,Molecule ChEMBL ID,Standard Type,Standard Relation,Standard Value,Standard Units,pChEMBL Value,Assay ChEMBL ID,Target ChEMBL ID,Target Name,Target Organism,Target Type,Document ChEMBL ID,IC50_nM,pIC50,Smiles
0,CHEMBL4443947,IC50,'=',0.022,nM,10.66,CHEMBL4361896,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4359855,0.022,10.66,CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)CC...
1,CHEMBL4556091,IC50,'=',0.026,nM,10.59,CHEMBL4345131,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4342485,0.026,10.59,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
2,CHEMBL4566431,IC50,'=',0.078,nM,10.11,CHEMBL4345131,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4342485,0.078,10.11,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
3,CHEMBL4545898,IC50,'=',0.081,nM,10.09,CHEMBL4345131,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4342485,0.081,10.09,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
4,CHEMBL4448950,IC50,'=',0.099,nM,10.00,CHEMBL4361896,CHEMBL3778,Interleukin-1 receptor-associated kinase 4,Homo sapiens,SINGLE PROTEIN,CHEMBL4359855,0.099,10.00,COc1cc2c(OC[C@@H]3CCC(=O)N3)ncc(C#CCCCCCCCCCCC...


In [19]:
train = pd.concat([data_train['Molecule ChEMBL ID'], data_train['Smiles']], axis=1)

In [20]:
train.head()

,Molecule ChEMBL ID,Smiles
0,CHEMBL4443947,CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)CC...
1,CHEMBL4556091,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
2,CHEMBL4566431,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
3,CHEMBL4545898,CC(C)(O)[C@H](F)CN1Cc2cc(NC(=O)c3cnn4cccnc34)c...
4,CHEMBL4448950,COc1cc2c(OC[C@@H]3CCC(=O)N3)ncc(C#CCCCCCCCCCCC...


In [21]:
test_smile = train['Smiles'][0]
test_smile

'CN[C@@H](C)C(=O)N[C@H](C(=O)N1C[C@@H](NC(=O)CCOCCOCCOCC#Cc2cnc(OC[C@@H]3CCC(=O)N3)c3cc(OC)c(C(N)=O)cc23)C[C@H]1C(=O)N[C@@H]1CCCc2ccccc21)C1CCCCC1'

In [64]:
smiles = train['Smiles'].values

In [61]:
# INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
sentence = 'Chúng_tôi là những nghiên_cứu_viên .'  

input_ids = torch.tensor([tokenizer.encode(test_smile, max_length=256, truncation=True)])

with torch.no_grad():
    features = model(input_ids)

In [68]:
input_ids.shape[1]

119

In [79]:
input_ids = torch.tensor([tokenizer.encode(smiles[4], max_length=256, truncation=True)])
input_ids.shape

torch.Size([1, 112])

In [84]:
max_len = -1
for s in smiles:
    input_ids = torch.tensor([tokenizer.encode(s)])
    
    size = input_ids.shape[1]
    if size > max_len:
        print(size)
        max_len = size

119


In [82]:
features[0].shape

torch.Size([1, 119, 384])

In [12]:
features

MaskedLMOutput(loss=None, logits=tensor([[[-2.0801, -1.5884, -2.7264,  ..., -0.8330, -2.2768, -3.9664],
         [-2.0189, -1.8411, -1.1740,  ..., -0.8589, -3.1837, -3.6122],
         [-1.5233, -1.9519, -1.5481,  ..., -1.4673, -1.9218, -1.9707],
         ...,
         [-2.1487, -2.0737, -2.8245,  ..., -1.9424, -2.2815, -3.8734],
         [-2.7176, -2.2366, -3.3407,  ..., -2.7212, -1.4406, -2.9181],
         [-2.5865, -1.8417, -2.0097,  ..., -1.0150, -2.0509, -3.7648]]]), hidden_states=None, attentions=None)

In [28]:
torch.mean(features[0], 1).reshape(-1).shape

torch.Size([767])

In [25]:
class AttentionPooling(nn.Module):
    def __init__(self, hidden_size):
        super(AttentionPooling, self).__init__()
        self.attention = nn.Linear(hidden_size, 1)

    def forward(self, outputs):
        weights = torch.softmax(self.attention(outputs), dim=1)
        context_vector = torch.sum(weights * outputs, dim=1)
        return context_vector

In [28]:
attention = AttentionPooling(hidden_size=600)

In [30]:
attention(features[0]).shape

torch.Size([1, 600])

In [52]:
# Load model directly
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("E://competition//atoms//ChemBERTa-77M-MTR")
model = AutoModel.from_pretrained("E://competition//atoms//ChemBERTa-77M-MTR")

c:\Users\Admin\miniconda3\envs\crawl\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at E://competition//atoms//ChemBERTa-77M-MTR and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
inputs = tokenizer.encode_plus(
            test_smile,
            None,
            add_special_tokens=True,
            max_length=64,
            padding='max_length',
            return_token_type_ids=False,
            truncation=True)
input_ids = torch.tensor(inputs['input_ids'], dtype=torch.long)
attention_mask = torch.tensor(inputs['attention_mask'], dtype=torch.long)
with torch.no_grad():
    features = model(input_ids=input_ids.unsqueeze(0), attention_mask=attention_mask.unsqueeze(0))

In [55]:
features[0].shape

torch.Size([1, 64, 384])